In [ ]:
import tkinter as tk
from tkinter import messagebox
import random

EMPTY = ' '
PLAYER_X = 'X'
PLAYER_O = 'O'

class TicTacToeGUI:
    def __init__(self, root):
        self.root = root
        self.root.title("Tic Tac Toe")
        self.root.geometry("400x350")  
        self.board = [EMPTY] * 9
        self.buttons = []
        self.current_player = PLAYER_X
        self.play_with_computer = False
        self.human_player = PLAYER_X
        self.ai_player = PLAYER_O
        self.create_start_screen()

    def create_start_screen(self):
        self.clear_window()
        tk.Label(self.root, text="Tic Tac Toe", font=('Helvetica', 20)).pack(pady=10)
        tk.Button(self.root, text="2 Player Game", command=self.start_two_player).pack(pady=5)
        tk.Button(self.root, text="Play vs Computer", command=self.start_vs_computer).pack(pady=5)

    def start_two_player(self):
        self.play_with_computer = False
        self.reset_board()
        self.create_board()

    def start_vs_computer(self):
        self.play_with_computer = True
        self.choose_symbol_screen()

    def choose_symbol_screen(self):
        self.clear_window()
        tk.Label(self.root, text="Choose your symbol:", font=('Helvetica', 14)).pack(pady=10)
        tk.Button(self.root, text="X (You go first)", command=lambda: self.set_symbol(PLAYER_X)).pack(pady=5)
        tk.Button(self.root, text="O (Computer goes first)", command=lambda: self.set_symbol(PLAYER_O)).pack(pady=5)

    def set_symbol(self, symbol):
        self.human_player = symbol
        self.ai_player = PLAYER_O if symbol == PLAYER_X else PLAYER_X
        self.current_player = PLAYER_X
        self.reset_board()
        self.create_board()
        if self.ai_player == PLAYER_X:
            self.root.after(500, self.computer_move)

    def create_board(self):
        self.clear_window()
        self.buttons = []
    
        # Configure the 3x3 grid to expand with window size
        for i in range(3):
            self.root.grid_rowconfigure(i, weight=1)
            self.root.grid_columnconfigure(i, weight=1)
    
        for i in range(9):
            btn = tk.Button(
                self.root,
                text='',
                font=('Helvetica', 32),  # Larger font scales better
                command=lambda i=i: self.on_button_click(i)
            )
            btn.grid(row=i//3, column=i%3, sticky="nsew", padx=2, pady=2)
            self.buttons.append(btn)


    def on_button_click(self, index):
        if self.board[index] != EMPTY:
            return
        if self.play_with_computer and self.current_player != self.human_player:
            return

        self.make_move(index, self.current_player)

        winner = self.check_winner()
        if winner:
            self.game_over(f"{winner} wins!")
        elif self.check_draw():
            self.game_over("It's a draw!")
        else:
            self.switch_player()
            if self.play_with_computer and self.current_player == self.ai_player:
                self.root.after(500, self.computer_move)

    def make_move(self, index, player):
        self.board[index] = player
        self.buttons[index].config(text=player)

    def computer_move(self):
        best_score = -float('inf')
        best_move = None
        for i in range(9):
            if self.board[i] == EMPTY:
                self.board[i] = self.ai_player
                score = self.minimax(0, False)
                self.board[i] = EMPTY
                if score > best_score:
                    best_score = score
                    best_move = i

        if best_move is not None:
            self.make_move(best_move, self.ai_player)

            winner = self.check_winner()
            if winner:
                self.game_over(f"{winner} wins!")
            elif self.check_draw():
                self.game_over("It's a draw!")
            else:
                self.switch_player()

    def minimax(self, depth, is_maximizing):
        winner = self.check_winner()
        if winner == self.ai_player:
            return 1
        elif winner == self.human_player:
            return -1
        elif self.check_draw():
            return 0

        if is_maximizing:
            best_score = -float('inf')
            for i in range(9):
                if self.board[i] == EMPTY:
                    self.board[i] = self.ai_player
                    score = self.minimax(depth + 1, False)
                    self.board[i] = EMPTY
                    best_score = max(score, best_score)
            return best_score
        else:
            best_score = float('inf')
            for i in range(9):
                if self.board[i] == EMPTY:
                    self.board[i] = self.human_player
                    score = self.minimax(depth + 1, True)
                    self.board[i] = EMPTY
                    best_score = min(score, best_score)
            return best_score

    def switch_player(self):
        self.current_player = PLAYER_O if self.current_player == PLAYER_X else PLAYER_X

    def check_winner(self):
        for a, b, c in [(0,1,2),(3,4,5),(6,7,8),(0,3,6),(1,4,7),(2,5,8),(0,4,8),(2,4,6)]:
            if self.board[a] == self.board[b] == self.board[c] != EMPTY:
                return self.board[a]
        return None

    def check_draw(self):
        return EMPTY not in self.board and not self.check_winner()

    def game_over(self, message):
        messagebox.showinfo("Game Over", message)
        self.create_start_screen()

    def reset_board(self):
        self.board = [EMPTY] * 9

    def clear_window(self):
        for widget in self.root.winfo_children():
            widget.destroy()

if __name__ == '__main__':
    root = tk.Tk()
    game = TicTacToeGUI(root)
    root.mainloop()